In [1]:
import pandas as pd

# 원본 CSV 경로
input_path = "/workspace/Yujin/GIA/data/Final/dataset_for_evaluation_fin.csv"
# 저장할 CSV 경로
output_path = "/workspace/Yujin/GIA/Network-Management-System-main/dataset/test_fin.csv"

# CSV 읽기
df = pd.read_csv(input_path)

# 필요한 컬럼만 추출
df_selected = df[["origin","question", "ground_truth", "explanation"]]

# 새 CSV로 저장
df_selected.to_csv(output_path, index=False, encoding="utf-8")

print(f"✅ Saved test.csv with {len(df_selected)} rows at {output_path}")


✅ Saved test.csv with 813 rows at /workspace/Yujin/GIA/Network-Management-System-main/dataset/test_fin.csv


In [13]:
!pip install langchain-unstructured 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 11.4 MB/s  0:00:01 eta 0:00:01
  Attempting uninstall: onnxruntime
    Found existing installation: onnxruntime 1.22.0
    Uninstalling onnxruntime-1.22.0:
      Successfully uninstalled onnxruntime-1.22.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [langchain-unstructured]


In [14]:
# pip install -U langchain-unstructured unstructured
from langchain_unstructured import UnstructuredLoader

loader = UnstructuredLoader(
    "/workspace/jke/xml_parssing/ce1.xml",
    chunking_strategy="by_title",   # 또는 "basic"
    max_characters=4000,
    combine_text_under_n_chars=1200,
    include_orig_elements=False,
)
docs = loader.load()


In [ ]:
# --- Settings ---
from pathlib import Path
from datetime import datetime
import re

# Where to save .txt files
OUT_DIR = Path("/workspace/jke/xml_parssing/txt_out")  # change if you like
OUT_DIR.mkdir(parents=True, exist_ok=True)

# (Optional) original XML path for metadata fallback
XML_PATH = "/workspace/jke/xml_parssing/ce1.xml"

def _slug(s: str, default: str = "element"):
    s = (s or "").strip()
    if not s:
        return default
    s = re.sub(r"[^\w\-\.]+", "_", s, flags=re.U)
    s = re.sub(r"_+", "_", s)
    return s.strip("_")[:64] or default

def _pick_section(md: dict, idx: int):
    # Try to create a human-ish "section" label from metadata
    return (
        md.get("xpath")
        or md.get("tag")
        or md.get("type")
        or md.get("element_id")
        or f"element_{idx}"
    )

def _format_doc(doc, idx: int, source_xml: str) -> str:
    md = getattr(doc, "metadata", {}) or {}
    content = (getattr(doc, "page_content", "") or "").strip()

    section = _pick_section(md, idx)
    device = md.get("device_name") or md.get("device") or ""   # use if present
    source = md.get("source") or source_xml
    last_mod = md.get("last_modified") or ""

    # Your requested style:
    # [METADATA] ...   [CONTENT] ...   [SECTION] ...
    txt = []
    txt.append("[METADATA]")
    if section:     txt.append(f"section: {section}")
    if device:      txt.append(f"device_name: {device}")
    if source:      txt.append(f"source: {source}")
    if last_mod:    txt.append(f"last_modified: {last_mod}")
    txt.append(f"exported_at: {datetime.now().isoformat(timespec='seconds')}")
    txt.append("")
    txt.append("[CONTENT]")
    txt.append(f"[SECTION] {section}")
    txt.append(content if content else "<EMPTY>")
    return "\n".join(txt).rstrip() + "\n"

def save_docs_as_txt(docs, out_dir: Path, source_xml: str = "") -> list[Path]:
    paths = []
    base = _slug(Path(source_xml).stem if source_xml else "docs")
    for i, doc in enumerate(docs, start=1):
        md = getattr(doc, "metadata", {}) or {}
        sect = _pick_section(md, i)
        fname = f"{base}_{i:04d}__{_slug(sect)}.txt"
        p = out_dir / fname
        p.write_text(_format_doc(doc, i, source_xml), encoding="utf-8")
        paths.append(p)
    return paths

def save_combined(docs, out_dir: Path, source_xml: str = "") -> Path:
    base = _slug(Path(source_xml).stem if source_xml else "docs")
    p = out_dir / f"{base}__combined.txt"
    parts = [_format_doc(doc, i, source_xml) for i, doc in enumerate(docs, start=1)]
    p.write_text("\n" + ("\n" + ("-"*80) + "\n").join(parts) + "\n", encoding="utf-8")
    return p

# --- Run export ---
file_paths = save_docs_as_txt(docs, OUT_DIR, XML_PATH)
combined_path = save_combined(docs, OUT_DIR, XML_PATH)

print(f"✅ Saved {len(file_paths)} files to: {OUT_DIR}")
print("First 3 files:")
for p in file_paths[:3]:
    print(" -", p)
print("Combined file:", combined_path)


✅ Saved 4 files to: /workspace/jke/xml_parssing/txt_out
First 3 files:
 - /workspace/jke/xml_parssing/txt_out/ce1_0001__44afeec75c313b27327473795fe940b8.txt
 - /workspace/jke/xml_parssing/txt_out/ce1_0002__9521cc5c5694bcdec146920f0aed757f.txt
 - /workspace/jke/xml_parssing/txt_out/ce1_0003__a7f981ed8ac0f37653e97dc0d54de863.txt
Combined file: /workspace/jke/xml_parssing/txt_out/ce1__combined.txt
